<a href="https://colab.research.google.com/github/Aryu-Tamura/GoogleColab_ChatbotTest/blob/main/01_%E8%8B%B1%E4%BC%9A%E8%A9%B1%E3%83%81%E3%83%A3%E3%83%83%E3%83%88%E3%83%9C%E3%83%83%E3%83%88_MVP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**MVP1: シンプルなテキスト英会話ボット**

目的: テキストでAI（英語教師）と簡単な会話ができる最小限のWebアプリを作ります。（AIへの指示は日本語、例文は英語）

手順:

1. 必要なライブラリをインストールします。
2. ライブラリを読み込みます。
3. OpenAI APIキーを設定します。
4. まずは関数化せずに、直接APIを呼び出して動作を確認します。
5. PI呼び出し部分を関数にします。
6. 作成した関数をテストします。
7. Gradioを使ってWebアプリの見た目（UI）を作ります。
8. Webアプリを起動します。

------------------------------


1. ライブラリのインストール

（このセルを実行すると、プログラムに必要な部品（ライブラリ）が準備されます）

In [ ]:
!pip install openai
!pip install gradio

------------------

2. ライブラリのインポート

（インストールした部品を使えるように読み込みます）

In [ ]:
import gradio as gr
import openai

----------------------


3. OpenAI APIキーの設定

（OpenAIのサービスを使うための「鍵」を設定します。ColabのSecrets機能を使うのが安全です）
※事前にColabの左メニュー(🔑)で `OPENAI_API_KEY` という名前でAPIキーを設定しておいてください。



In [ ]:
from google.colab import userdata # APIキーを安全に読み込むため
key = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=key)

---------------------


4. API直接呼び出しテスト

（関数を作る前に、直接APIを呼び出して、AIが応答するか試してみます）

4-1. AIへの指示（システムプロンプト）とユーザー入力を設定

In [ ]:
# AIの役割や話し方を指示します (システムプロンプト：指示は日本語、例文は英語)
system_prompt = """あなたは、ユーザーと自然で流暢な英会話を行うAI英語教師です。

# あなたの役割
- ユーザーの英語力向上のために、自然で日常的な英会話を行ってください。
- ユーザーの英語表現や文法に間違いがあれば、会話の流れを止めないように、自然な形で指摘し、より適切な表現を提案してください。
- ユーザーの質問や発言には、共感的に、そして前向きな態度で応答してください。

# 回答の形式
- 会話は常に英語で行ってください。日本語で応答してはいけません。
- 回答は、原則として短く（1～3文程度）、自然な口語表現を使ってください。
- ユーザーが文法や表現のミスをした場合は、以下のような形式で訂正例を示してください。
  例 (Example): User: "I goed to park yesterday." AI: "Oh, nice! You mean, 'I went to the park yesterday,' right?"

# 入出力の例 (Input/Output Examples)
User: "Hello! How was your day?" AI: "Hey! My day's going great, thanks for asking. How about you?"
User: "I'm study English today." AI: "That's great! Just a quick tip: you'd say, 'I'm studying English today.' What are you working on right now?"
User: "Can you help me about grammar?" AI: "Of course! I'd say, 'Can you help me with grammar?' Sure thing! What grammar point do you want to talk about?" """

# ユーザーがAIに話しかける内容 (ユーザー入力)
user_input = "Hello! I want to practice English conversation."

# APIに渡すメッセージリストを作成 (システム指示 + ユーザー入力)
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_input}
]

print("AIへの指示とユーザー入力の準備完了。")

4-2. OpenAI APIを呼び出して応答を取得



In [ ]:
# OpenAI API (Chat Completions) を呼び出します
response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # 使用するAIモデル
    messages=messages,      # 上で準備したメッセージリスト
    max_tokens=100,         # 応答の長さの上限（だいたい）
    temperature=0.7         # 応答の多様性（0に近いほど毎回同じ、1に近いほど変化）
)

print("APIから応答を受け取りました。")

4-3. API応答の中身を確認



In [ ]:
# 応答の中から、AIが生成したメッセージ部分を取り出す
ai_message_content = response.choices[0].message.content
print(ai_message_content) #user_inputに対する返答

---------------------


5. 関数化

（API呼び出しのコードを、使いやすいように一つの関数にまとめます）

In [ ]:
def english_teacher_bot(user_input):
    """
    ユーザーからの入力(user_input)を受け取り、
    英語教師AIとして応答を返す関数。
    """
    # AIの役割設定 (システムプロンプト：指示は日本語、例文は英語) - 関数の中で定義
    system_prompt = """あなたは、ユーザーと自然で流暢な英会話を行うAI英語教師です。

# あなたの役割
- ユーザーの英語力向上のために、自然で日常的な英会話を行ってください。
- ユーザーの英語表現や文法に間違いがあれば、会話の流れを止めないように、自然な形で指摘し、より適切な表現を提案してください。
- ユーザーの質問や発言には、共感的に、そして前向きな態度で応答してください。

# 回答の形式
- 会話は常に英語で行ってください。日本語で応答してはいけません。
- 回答は、原則として短く（1～3文程度）、自然な口語表現を使ってください。
- ユーザーが文法や表現のミスをした場合は、以下のような形式で訂正例を示してください。
  例 (Example): User: "I goed to park yesterday." AI: "Oh, nice! You mean, 'I went to the park yesterday,' right?"

# 入出力の例 (Input/Output Examples)
User: "Hello! How was your day?" AI: "Hey! My day's going great, thanks for asking. How about you?"
User: "I'm study English today." AI: "That's great! Just a quick tip: you'd say, 'I'm studying English today.' What are you working on right now?"
User: "Can you help me about grammar?" AI: "Of course! I'd say, 'Can you help me with grammar?' Sure thing! What grammar point do you want to talk about?" """

    # APIに渡すメッセージリストを作成
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input} # 関数の引数で受け取ったユーザー入力を使う
    ]

    # OpenAI APIを呼び出す
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=100,
        temperature=0.7
    )

    # AIの応答テキストを関数の結果として返す
    return response.choices[0].message.content

print("関数 'english_teacher_bot' を定義しました。")

----------------------------

6. 関数テスト

（作成した関数が正しく動作するか、いくつかの入力で試してみます）

In [ ]:
# テスト1
response1 = english_teacher_bot("Hi! How's it going?")
print(f"入力1: Hi! How's it going?")
print(f"応答1: {response1}\n")

In [ ]:
# テスト2
response2 = english_teacher_bot("I learned new vocabulary yesterday.")
print(f"入力2: I learned new vocabulary yesterday.")
print(f"応答2: {response2}")

------------------------------


7. Gradio Interface 作成

Gradioのコンセプトは「関数」を「GUI化」すること。   
作成した関数を使って、Webアプリの画面（インターフェース）を作ります

In [ ]:
# gr.Interface を使ってUIを作成します
bot_interface = gr.Interface(
    fn=english_teacher_bot,  # このUIで使う関数を指定
    inputs="text",           # 入力はテキストボックス (シンプル指定)
    outputs="text",          # 出力もテキストボックス (シンプル指定)
    title="英会話チャットボット", # アプリのタイトル
    description="英語で話しかけると、AI先生が返事してくれます。" # 簡単な説明
)

print("Gradioインターフェースを作成しました。")

--------------------------------

8. Webアプリの起動

作成したインターフェースを起動して、Webアプリとして使えるようにします。

In [ ]:
# 作成したUIを起動します
# share=True にすると、一時的に外部からアクセスできるURLも生成されます
bot_interface.launch(share=True)